In [1]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
import matplotlib.pyplot as plt
from torchvision import datasets, transforms
import torchvision.models as models

In [2]:
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize(mean=0.5, std=0.5),
                              ])

trainset = datasets.FashionMNIST('F_MNIST_data/', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = datasets.FashionMNIST('F_MNIST_data/', download=True, train=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=True)

for images, labels in trainloader:
    print(type(images))
    print(images.shape)
    print(labels.shape)
    break

<class 'torch.Tensor'>
torch.Size([64, 1, 28, 28])
torch.Size([64])


In [3]:
model_old =  nn.Sequential(nn.Dropout(p=0.2),
                       nn.Linear(784, 512),
                       nn.ReLU(),
                       nn.Dropout(p=0.2),
                       nn.Linear(512, 128),
                       nn.ReLU(),
                       nn.Dropout(p=0.2),
                       nn.Linear(128, 64),
                       nn.ReLU(),
                       nn.Linear(64,10),
                       nn.LogSoftmax(dim=1))



In [5]:
model = models.densenet121(pretrained=True)

for param in model.parameters():
    param.requires_grad = False

from collections import OrderedDict

classifier = nn.Sequential(OrderedDict([
    ('fcl', nn.Linear(1024,512)),
    ('relu', nn.ReLU()),
    ('fcl', nn.Linear(512,10)),
    ('output', nn.LogSoftmax(dim=1)),
]))

model.classifier = classifier
model

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [6]:
criterion = nn.CrossEntropyLoss()

In [7]:
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [8]:
epochs=10

In [9]:
train_losses = []
test_losses = []

for e in range(epochs):
    running_loss = 0
    for images, labels in trainloader:
        images = images.view(images.shape[0], -1)
        optimizer.zero_grad()
        output = model.forward(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    test_loss = 0
    accuracy = 0
    with torch.no_grad():
        for test_images, test_labels in testloader:
            test_images = test_images.view(test_images.shape[0], -1)
            test_output = model.forward(test_images)
            test_loss = criterion(test_output, test_labels)
               
            proba = torch.exp(test_output)
#            top_proba, top_class = proba.topk(1, dim=1)
#            okclass = top_class == labels.view(*top_class.shape)
#            accuracy += torch.mean(equals.type(okclass.type(torch.FloatTensor)))
    train_losses.append(running_loss/len(trainloader))
    test_losses.append(test_loss/len(testloader))
    print(f"epoch: {e+1}")
    print(f"training loss: {running_loss/len(trainloader)}")       
    print(f"test loss: {running_loss/len(testloader)}")
#    print(f"test accuracy: {accuracy/len(testloader)}")


RuntimeError: Expected 4-dimensional input for 4-dimensional weight [64, 3, 7, 7], but got 2-dimensional input of size [64, 784] instead

In [ ]:
## makes the kernel bug
#%matplotlib inline
#%config InlineBackend.figure_format = 'retina'

#plt.plot(train_losses, label='train loss')
#plt.plot(test_losses, label='validation loss')
#plt.legend(framon=False)
#plt.legend(framon=True)

In [ ]:
model.state_dict().keys()

In [ ]:
torch.save(model.state_dict(), "g:\projets\state.pth" )

In [ ]:
model.load_state_dict(torch.load("g:\projets\state.pth"))
model.eval()